# **Imports**🎇

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import os
import json
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, random_split, Dataset
import torch.optim as optim

import torchvision
from torchvision import transforms as T
from torchvision.utils import make_grid
from torchvision.transforms import functional as F

# **Utils**

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

n_epoch = 50

# **Prepare Labels and Files**

In [9]:
# !labelme

In [10]:
# for folder in ['train', 'valid', 'test']:
#     for file in os.listdir(os.path.join(folder, 'photo/')):
#         filename = file.split('.')[0]+'.json'
#         existing_filepath = os.path.join('labels', filename)
#         if os.path.exists(existing_filepath):
#             new_filepath = os.path.join(folder, 'label', filename)
#             os.replace(existing_filepath, new_filepath)


# **Functions**

In [25]:
class TrumpDataset(Dataset):
    def __init__(self, phase):
        self.phase = phase

        self.images_list = []
        self.labels = []
        for item in os.listdir(os.path.join(phase, 'photo')):
            img_path = os.path.join(phase, 'photo', item)
            self.images_list.append(img_path)
            self.labels.append(os.path.join(phase, 'label', '.'.join(item.split('.')[:-1]) + '.json'))


    def __getitem__(self, item):
        img_path = os.path.join(self.images_list[item])
        img = Image.open(img_path).convert('RGB')
        img = F.to_tensor(img)
          
        with open(self.labels[item]) as f:
            load = json.load(f)
            points = []
            for i in range(len(load['shapes'])):
                points.append(
                    [load['shapes'][i]['points'][1][0],
                     load['shapes'][i]['points'][0][1],
                     load['shapes'][i]['points'][0][0],
                     load['shapes'][i]['points'][1][1]])

        boxes = torch.tensor(points, dtype=torch.float32)
        labels = torch.ones((len(load['shapes']), ), dtype=torch.int64)
        #
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        #
        return img, target

    def __len__(self):
        return len(self.images_list)

In [36]:
import math
def train_one_epoch(model, optimizer, train_loader, epoch=None):
    model.train()
    total_loss = 0
    with tqdm(train_loader, unit="batch") as tepoch:
        for images, targets in tepoch:
            if epoch is not None:
                tepoch.set_description(f"Epoch {epoch}")
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            total_loss += losses
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
    return total_loss/len(train_dataloader)

# **Dataloader**⚽

In [27]:
train = TrumpDataset('train')
valid = TrumpDataset('valid')
test = TrumpDataset('test')

In [28]:
def new_concat(batch):
  return tuple(zip(*batch))

In [29]:
train_loader = DataLoader(train,
                            batch_size=2,
                            shuffle=True,
                            collate_fn=new_concat)
valid_loader = DataLoader(valid,
                            batch_size=1,
                            shuffle=True,
                            collate_fn=new_concat)
test_loader = DataLoader(test,
                            batch_size=1,
                            shuffle=True,
                            collate_fn=new_concat)


# **Model**🎯

In [31]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(1024, 2)
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [32]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# **Train**🎢

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    loss = train_one_epoch(model, optimizer, train_loader)
    print('epoch [{}]:  \t lr: {}  \t loss: {}  '.format(epoch, lr_scheduler.get_last_lr(), loss))
    lr_scheduler.step()

 98%|█████████▊| 44/45 [09:16<00:12, 12.91s/batch]